In [1]:
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns; sns.set()
%matplotlib inline

import keras
from keras import objectives
from keras.callbacks import EarlyStopping
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Input, Multiply, Add
from keras.optimizers import Adam, Nadam
import salty
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from random import shuffle
import pandas as pd
import random

#Keras build
from keras import backend as K
from keras.objectives import binary_crossentropy #objs or losses
from keras.models import Model
from keras.layers import Input, Dense, Lambda, Layer
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#cation data
cations = pd.read_csv('../data/cations.csv')
cations = cations['smiles_string']
salts = pd.read_csv('../data/salts.csv')
salts = salts['smiles_string']
categories = pd.read_csv('../data/categories.csv')
categories = categories['category']
coldic = pd.read_csv('../data/coldic.csv')
coldic = coldic.to_dict(orient='records')[0]
salt_coldic = pd.read_csv('../data/salt_coldic.csv')
salt_coldic = salt_coldic.to_dict(orient='records')[0]
salt_categories = pd.read_csv('../data/salt_categories.csv')
salt_categories = salt_categories['category']
density_coldic = pd.read_csv('../data/density_coldic.csv')
density_coldic = density_coldic.to_dict(orient='records')[0]
density_categories = pd.read_csv('../data/density_categories.csv')
density_categories = density_categories['category']

#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

#training array info
smile_max_length = 105
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

/home/wesley/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#plot
import matplotlib.pylab as plt
import seaborn as sns; sns.set()
from matplotlib import colors
from itertools import cycle

#data
import pandas as pd
import numpy as np
from numpy.linalg import norm
import json
import random
import copy

#ML
import keras
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import backend as K
from keras import objectives
from keras.objectives import binary_crossentropy #objs or losses
from keras.layers import Dense, Dropout, Input, Multiply, Add, Lambda
from keras.layers.core import Dense, Activation, Flatten, RepeatVector
from keras.layers.wrappers import TimeDistributed
from keras.layers.recurrent import GRU
from keras.layers.convolutional import Convolution1D

#chem
import salty
from rdkit import Chem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit import DataStructs
from rdkit.Chem import Draw


class MoleculeVAE():

    autoencoder = None
    
    def create(self,
               charset,
               max_length = 105,
               latent_rep_size = 292,
               weights_file = None,
               qspr = False,
               conv_layers=3,
               gru_layers=3,
               conv_filters=3, 
               conv_kernel_size=9,
               gru_output_units=501):
        charset_length = len(charset)
    
        x = Input(shape=(max_length, charset_length))
        _, z = self._buildEncoder(x, 
                                  latent_rep_size, 
                                  max_length,
                                  conv_layers, 
                                  conv_filters, 
                                  conv_kernel_size)
        self.encoder = Model(x, z)
        encoded_input = Input(shape=(latent_rep_size,))
        self.decoder = Model(
            encoded_input,
            self._buildDecoder(
                encoded_input,
                latent_rep_size,
                max_length,
                charset_length,
                gru_layers,
                gru_output_units
            )
        )

        x1 = Input(shape=(max_length, charset_length))
        vae_loss, z1 = self._buildEncoder(x1, 
                                          latent_rep_size, 
                                          max_length,
                                          conv_layers, 
                                          conv_filters,
                                          conv_kernel_size)
        
        if qspr:
            self.autoencoder = Model(
                x1,
                self._buildDecoderQSPR(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )

        else:
            self.autoencoder = Model(
                x1,
                self._buildDecoder(
                    z1,
                    latent_rep_size,
                    max_length,
                    charset_length,
                    gru_layers,
                    gru_output_units
                )
            )
            
        self.qspr = Model(
            x1,
            self._buildQSPR(
                z1,
                latent_rep_size,
                max_length,
                charset_length
            )
        )


        if weights_file:
            self.autoencoder.load_weights(weights_file, by_name = True)
            self.encoder.load_weights(weights_file, by_name = True)
            self.decoder.load_weights(weights_file, by_name = True)
            self.qspr.load_weights(weights_file, by_name = True)
        if qspr:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss, 'qspr': 'mean_squared_error'},
                                     metrics = ['accuracy', 'mse'])
        else:
            self.autoencoder.compile(optimizer = 'Adam',
                                     loss = {'decoded_mean': vae_loss},
                                     metrics = ['accuracy'])
    def _buildEncoder(self, x, latent_rep_size, max_length, epsilon_std = 0.01, conv_layers=3, conv_filters=9, conv_kernel_size=9):
        h = Convolution1D(9, 9, activation = 'relu', name='conv_1')(x)
        for convolution in range(conv_layers-2):
            h = Convolution1D(conv_filters, conv_kernel_size, activation = 'relu', name='conv_{}'.format(convolution+2))(h)
        h = Convolution1D(10, 11, activation = 'relu', name='conv_{}'.format(conv_layers))(h)
        h = Flatten(name='flatten_1')(h)
        h = Dense(435, activation = 'relu', name='dense_1')(h)

        def sampling(args):
            z_mean_, z_log_var_ = args
            batch_size = K.shape(z_mean_)[0]
            epsilon = K.random_normal(shape=(batch_size, latent_rep_size), mean=0., stddev = epsilon_std)
            return z_mean_ + K.exp(z_log_var_ / 2) * epsilon

        z_mean = Dense(latent_rep_size, name='z_mean', activation = 'linear')(h)
        z_log_var = Dense(latent_rep_size, name='z_log_var', activation = 'linear')(h)

        def vae_loss(x, x_decoded_mean):
            x = K.flatten(x)
            x_decoded_mean = K.flatten(x_decoded_mean)
            xent_loss = max_length * objectives.binary_crossentropy(x, x_decoded_mean)
            kl_loss = - 0.5 * K.mean(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis = -1)
            return xent_loss + kl_loss

        return (vae_loss, Lambda(sampling, output_shape=(latent_rep_size,), name='lambda')([z_mean, z_log_var]))

    def _buildDecoderQSPR(self, z, latent_rep_size, max_length, charset_length, gru_layers=3):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(501, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        h = Dense(latent_rep_size, name='qspr_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        smiles_qspr = Dense(1, activation='linear', name='qspr')(h)

        return smiles_decoded, smiles_qspr

    def _buildDecoder(self, z, latent_rep_size, max_length, charset_length, gru_layers=3, gru_output_units=501):

        h = Dense(latent_rep_size, name='latent_input', activation = 'relu')(z)
        h = RepeatVector(max_length, name='repeat_vector')(h)
        h = GRU(501, return_sequences = True, name='gru_1')(h)
        for gru in range(gru_layers-2):
            h = GRU(gru_output_units, return_sequences = True, name='gru_{}'.format(gru+2))(h)
        h = GRU(501, return_sequences = True, name='gru_{}'.format(gru_layers))(h)
        smiles_decoded = TimeDistributed(Dense(charset_length, activation='softmax'), name='decoded_mean')(h)

        return smiles_decoded
    
    def _buildQSPR(self, z, latent_rep_size, max_length, charset_length):
        h = Dense(latent_rep_size, name='latent_input', activation='relu')(z)
        h = Dense(100, activation='relu', name='hl_1')(h)
        h = Dropout(0.5)(h)
        return Dense(1, activation='linear', name='qspr')(h)

    def save(self, filename):
        self.autoencoder.save_weights(filename)
    
    def load(self, charset, weights_file, latent_rep_size = 292):
        self.create(charset, weights_file = weights_file, latent_rep_size = latent_rep_size)

In [3]:
vae = MoleculeVAE()
vae.create?

Signature: vae.create(charset, max_length=105, latent_rep_size=292, weights_file=None, qspr=False, conv_layers=3, gru_layers=3, conv_filters=9, conv_kernel_size=9, gru_output_units=501)
Docstring: <no docstring>
File:      /media/wesleybeckner/weshhd/wes/Dropbox/Python/py3/generative_learners_salts/notebooks/<ipython-input-2-837442963518>
Type:      method


In [15]:
vae.create(char_set, 
           max_length=51, 
           latent_rep_size=292,
           weights_file='1mil_GDB17_10.h5',
           qspr=False)

In [18]:
anions = []
for smi in salts:
    anions.append(smi.split('.')[1])

In [24]:
anions = []
for smi in salts:
    anions.append(smi.split('.')[1])
lens = []
for anion in anions:
    lens.append(len(anion))

In [25]:
max(lens)

62

In [39]:
#training data
#max anion len 62
#max cation len 48
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62, conv_layers=3, conv_filters=3, conv_kernel_size=9)

In [40]:
chemvae.autoencoder.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_47 (InputLayer)           (None, 62, 37)       0                                            
__________________________________________________________________________________________________
conv_1 (Conv1D)                 (None, 54, 9)        3006        input_47[0][0]                   
__________________________________________________________________________________________________
conv_2 (Conv1D)                 (None, 46, 9)        738         conv_1[0][0]                     
__________________________________________________________________________________________________
conv_3 (Conv1D)                 (None, 36, 10)       1000        conv_2[0][0]                     
__________________________________________________________________________________________________
flatten_1 

In [ ]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_62smi_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_62smi'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1768s 18ms/step - loss: 2.3151 - acc: 0.7021 - val_loss: 2.2591 - val_acc: 0.7072
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1768s 18ms/step - loss: 2.2513 - acc: 0.7070 - val_loss: 2.2269 - val_acc: 0.7091
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1766s 18ms/step - loss: 2.3051 - acc: 0.7029 - val_loss: 2.2560 - val_acc: 0.7078
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1769s 18ms/step - loss: 2.2421 - acc: 0.7072 - val_loss: 2.2396 - val_acc: 0.7079
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 1790s 18ms/step - loss: 2.2341 - acc: 0.7077 - val_loss: 2.2339 - val_acc: 0.7085
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [3]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_51smi_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_51smi'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2861s 29ms/step - loss: 2.3101 - acc: 0.6382 - val_loss: 2.2386 - val_acc: 0.6445
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2919s 29ms/step - loss: 2.2505 - acc: 0.6438 - val_loss: 2.2374 - val_acc: 0.6447
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
19392/99000 [====>.........................] - ETA: 39:25 - loss: 2.2434 - acc: 0.6443

KeyboardInterrupt: 

In [4]:
import json
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_51smi_gdbchar_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_51smi_gdbchar'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2949s 30ms/step - loss: 2.5879 - acc: 0.6383 - val_loss: 2.5128 - val_acc: 0.6441
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2948s 30ms/step - loss: 2.5196 - acc: 0.6439 - val_loss: 2.4958 - val_acc: 0.6471
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2999s 30ms/step - loss: 2.5283 - acc: 0.6434 - val_loss: 2.6054 - val_acc: 0.6415
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3029s 31ms/step - loss: 2.5299 - acc: 0.6430 - val_loss: 2.5025 - val_acc: 0.6460
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2976s 30ms/step - loss: 2.5269 - acc: 0.6434 - val_loss: 2.5163 - val_acc: 0.6444
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [5]:
#supporting functions
import sys
sys.path.insert(0, '../')
from scripts import *

In [6]:
import json
f = open("../data/gdb_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
#     chemvae.save('../models/1mil_GDB17_51smi_gdbchar_{}.h5'.format(p+1))
    
# with open('../models/history_{}.json'.format('1mil_GDB17_51smi_gdbchar'), 'w') as f:
#         json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 5646s 57ms/step - loss: 2.3607 - acc: 0.6542 - val_loss: 2.1776 - val_acc: 0.6833
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 5677s 57ms/step - loss: 1.6147 - acc: 0.7606 - val_loss: 1.3074 - val_acc: 0.8074
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 5702s 58ms/step - loss: 1.1788 - acc: 0.8273 - val_loss: 1.0910 - val_acc: 0.8419
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 5702s 58ms/step - loss: 0.9733 - acc: 0.8591 - val_loss: 0.9107 - val_acc: 0.8683
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 5758s 58ms/step - loss: 0.8342 - acc: 0.8805 - val_loss: 0.8692 - val_acc: 0.8787
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [8]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 51
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=51)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_51smi_saltchar_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_51smi_saltchar'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2848s 29ms/step - loss: 2.1012 - acc: 0.6518 - val_loss: 1.8331 - val_acc: 0.6810
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2861s 29ms/step - loss: 1.4858 - acc: 0.7541 - val_loss: 1.1758 - val_acc: 0.8094
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2858s 29ms/step - loss: 1.0317 - acc: 0.8357 - val_loss: 0.9028 - val_acc: 0.8567
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2845s 29ms/step - loss: 0.8354 - acc: 0.8680 - val_loss: 0.8972 - val_acc: 0.8608
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 2871s 29ms/step - loss: 0.7093 - acc: 0.8885 - val_loss: 0.6087 - val_acc: 0.9056
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [9]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.1000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62)
data_size = 100000
histories = []
for p in range(10):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    for i, smile in enumerate(padded_smiles[:data_size]):
#         linearly_scaled_prob = random.random() < 0.5#i/data_size
#         if linearly_scaled_prob:
#             smile = random.choice(cations)
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1

    X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_62smi_saltchar_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_62smi_saltchar'), 'w') as f:
        json.dump(histories, f)

Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3513s 35ms/step - loss: 2.1830 - acc: 0.7084 - val_loss: 2.0775 - val_acc: 0.7137
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3485s 35ms/step - loss: 1.7511 - acc: 0.7590 - val_loss: 1.5070 - val_acc: 0.7927
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3482s 35ms/step - loss: 1.4245 - acc: 0.8046 - val_loss: 1.3079 - val_acc: 0.8220
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3499s 35ms/step - loss: 1.2317 - acc: 0.8315 - val_loss: 1.1965 - val_acc: 0.8361
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
99000/99000 [==============================] - 3537s 36ms/step - loss: 1.5026 - acc: 0.7982 - val_loss: 1.6258 - val_acc: 0.7822
Train on 99000 samples, validate on 1000 samples
Epoch 1/1
9

In [21]:
anions = []
for anion in salts:
    anions.append(anion.split('.')[1])

In [22]:
anions = pd.DataFrame(anions)
anions.columns = ['smiles']
anions = anions.drop_duplicates()
anions = anions.reset_index(drop=True)
anions = anions['smiles']

In [26]:
chemvae = MoleculeVAE()

In [27]:
chemvae.create?

Signature: chemvae.create(charset, max_length=105, latent_rep_size=292, weights_file=None, qspr=False)
Docstring: <no docstring>
File:      ~/Dropbox/Python/py3/generative_learners_salts/scripts/vae.py
Type:      method


In [41]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.2000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62, weights_file='../models/1mil_GDB17_62smi_saltchar_10.h5')
data_size = 100000
histories = []
for p in range(5):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    padded_anions =  [pad_smiles(i, smile_max_length) for i in anions if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    anions_added = 0
    for i, smile in enumerate(padded_smiles):
        linearly_scaled_prob = random.random() < i/data_size
        if linearly_scaled_prob:
            smile = random.choice(padded_anions)
            anions_added += 1
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1
    print('anions added: {}'.format(anions_added))

#     X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_62smi_saltchar_500k_mix_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_62smi_saltchar_500k_mix'), 'w') as f:
        json.dump(histories, f)

anions added: 50098
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3681s 37ms/step - loss: 0.5418 - acc: 0.9314 - val_loss: 0.5815 - val_acc: 0.9245
anions added: 50081
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3617s 36ms/step - loss: 0.4996 - acc: 0.9353 - val_loss: 0.7252 - val_acc: 0.9139
anions added: 49991
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3614s 36ms/step - loss: 0.4834 - acc: 0.9375 - val_loss: 0.5096 - val_acc: 0.9335
anions added: 49955
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3606s 36ms/step - loss: 0.4477 - acc: 0.9420 - val_loss: 0.5087 - val_acc: 0.9343
anions added: 50138
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3615s 36ms/step - loss: 0.4

In [42]:
import json
f = open("../data/salt_char_to_index.json","r")
char_to_index = json.loads(f.read())
char_set = set(char_to_index.keys())
char_list = list(char_to_index.keys())
chars_in_dict = len(char_list)

#training array info
smile_max_length = 62
df = pd.read_csv('../data/GDB/GDB17.2000000', names=['smiles'])

#training data
chemvae = MoleculeVAE()
chemvae.create(char_set, max_length=62, weights_file='../models/1mil_GDB17_62smi_saltchar_500k_mix_5.h5')
data_size = 100000
histories = []
for p in range(5):
    values = df['smiles'][data_size*p:data_size*(p+1)]
    padded_smiles =  [pad_smiles(i, smile_max_length) for i in values if pad_smiles(i, smile_max_length)]
    padded_anions =  [pad_smiles(i, smile_max_length) for i in anions if pad_smiles(i, smile_max_length)]
    X_train = np.zeros((data_size, smile_max_length, chars_in_dict), dtype=np.float32)
    
    #for each i, randomly select whether to sample from GDB or cations (padded_smiles_2)
    anions_added = 0
    for i, smile in enumerate(padded_smiles):
#         linearly_scaled_prob = random.random() < i/data_size
#         if linearly_scaled_prob:
        smile = random.choice(padded_anions)
        anions_added += 1
        for j, char in enumerate(smile):
            X_train[i, j, char_to_index[char]] = 1
    print('anions added: {}'.format(anions_added))

#     X_train, X_test = train_test_split(X_train, test_size=0.01, random_state=42)   
    history = chemvae.autoencoder.fit(X_train, X_train, shuffle = False, validation_data=(X_test, X_test))
    histories.append(history.history)
    chemvae.save('../models/1mil_GDB17_62smi_saltchar_500k_mix_anion_{}.h5'.format(p+1))
    
with open('../models/history_{}.json'.format('1mil_GDB17_62smi_saltchar_500k_mix_anion'), 'w') as f:
        json.dump(histories, f)

anions added: 100000
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3601s 36ms/step - loss: 0.0491 - acc: 0.9958 - val_loss: 3.9338 - val_acc: 0.8143
anions added: 100000
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3581s 36ms/step - loss: 0.0395 - acc: 0.9965 - val_loss: 4.9384 - val_acc: 0.8041
anions added: 100000
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3548s 35ms/step - loss: 0.0835 - acc: 0.9918 - val_loss: 3.9590 - val_acc: 0.8081
anions added: 100000
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3552s 36ms/step - loss: 0.0826 - acc: 0.9918 - val_loss: 4.9319 - val_acc: 0.7929
anions added: 100000
Train on 100000 samples, validate on 1000 samples
Epoch 1/1
100000/100000 [==============================] - 3527s 35ms/step - loss

In [48]:
cationvae = MoleculeVAE()

In [49]:
cationvae.create?

Signature: cationvae.create(charset, max_length=105, latent_rep_size=292, weights_file=None, qspr=False)
Docstring: <no docstring>
File:      ~/Dropbox/Python/py3/generative_learners_salts/scripts/vae.py
Type:      method


In [53]:
#training array info
import json
f = open("../data/gdb_char_to_index.json","r")
cat_char_to_index = json.loads(f.read())
cat_char_set = set(cat_char_to_index.keys())
cat_char_list = list(cat_char_to_index.keys())
cat_chars_in_dict = len(cat_char_list)
cationvae.create(cat_char_set, max_length=51, weights_file='../models/1mil_GDB17_500K_mix_500K_cation_5.h5')

In [46]:
anionvae = chemvae

In [47]:
anionvae.autoencoder.layers